# Gamma

This notebook reproduces the salient characteristics of the [Gamma](https://dl.acm.org/doi/pdf/10.1145/3445814.3446702) accelerator.

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "..")

from src import utils

## Initialization

Initialize the input tensors. Tensor shapes and densities can be modified below.

**Warning:** Large tensors will overwhelm the video generation. Either:
1. Use small tensors; as a rule of thumb, fewer than 60 computes (e.g., multiplications) should be required.
2. Do not generate a video; remove the `spacetime` specification from the `mapping` before compiling.

In [ ]:
K = 10
M = 5
N = 6

density = [0.9, 0.5]
seed = 0

A_MK = Tensor.fromRandom(rank_ids=["M", "K"], shape=[M, K], seed=seed, density=density, name="A")
B_KN = Tensor.fromRandom(rank_ids=["K", "N"], shape=[K, N], seed=seed + 1, density=density, name="B")

## Compile and Run

Below is the TeAAL specification for Gamma. To simulate the accelerator:
1. Compile it to HiFiber by running the cell, inserting a new cell
2. Run the new cell, which will
    - Execute the kernel; multiplying the above defined matrices
    - Generate visualizations of the actions of the kernel

#### Notes
- Small tensors are required for video generation. Partition shapes are decreased accordingly for visualization purposes.
- If you are using large tensors, remove the spacetime specification to generate a kernel that does not produce videos. Outputs can still be checked below.

In [ ]:
yaml = """
einsum:
  declaration:
    A: [K, M]
    B: [K, N]
    T: [K, M, N]
    Z: [M, N]
  expressions:
    - T[k,m,n] = take(A[k,m], B[k,n], 1)
    - Z[m,n] = T[k,m,n] * A[k,m]
mapping:
  rank-order:
    A: [M, K]
    B: [K, N]
    T: [M, K, N]
    Z: [M, N]
  partitioning:
    T:
      M: [uniform_occupancy(A.2)]
      K: [uniform_occupancy(A.3)]
    Z:
      M: [uniform_occupancy(A.2)]
      K: [uniform_occupancy(A.3)]
  loop-order:
    T: [M1, M0, K1, K0, N]
    Z: [M1, M0, K1, N, K0]
  spacetime:
    T:
      space: [M0, K1]
      time: [M1, K0, N]
    Z:
      space: [M0, K1]
      time: [M1, N, K0]
"""

utils.compile(yaml)

## Check Results

Check that generated code computes the correct result.

**Note**: Should be used after compiling and running the kernel (above cell).

In [ ]:
utils.check_matmul(A_MK, B_KN, Z_MN)